In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime, timedelta
import os, sys
import urllib3
import time
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
sys.path.append(os.path.abspath(os.path.join(os.path.dirname("."), '..')))
from DataBase.DBUtilities import get_prices_from_db, insert_dataframe, ExecuteQuery, store_daily_data, validate_dataframe

In [2]:

# Liste des tickers que tu veux surveiller
tickers = ['ORGT', 'PALC', 'STAC', 'CFAC', 'ABJC', 'SIBC', 'ETIT', 'BOAB', 'UNLC']

# URL de base pour les données BRVM
url_price = 'https://www.brvm.org/fr/cours-actions/0'
url_per = 'https://www.brvm.org/fr/volumes/0'
url_capitalisation = 'https://www.brvm.org/fr/capitalisations/0'
# Faire la requête HTTP

def get_info(url):
    r = requests.get(url, verify=False)
    soup = BeautifulSoup(r.content, 'html.parser')
    #print(soup)
    rows = soup.select('table tbody tr')
    return rows

def get_col_from_rows(rows, col_id, date=None, exclude=["Valeur des transactions", "Capitalisation Actions", "Capitalisation des obligations"]):
    data = []
    date = datetime.today().strftime('%Y-%m-%d') if date is None else date
    for row in rows:
        cols = row.find_all('td')
        #print(cols, len(cols))
        if len(cols) < 5:
            continue
        code = cols[0].text.strip()
        if code in exclude:
            continue
        d = {"Ticker":code}
        for c in col_id:
            val = cols[c[1]].text.strip()
            try:
                val = val if c[2].lower() != 'numeric' else float(val.replace(',', '.').replace(' ', ''))
            except:
                #print(code, val)
                pass
            d[c[0]] = val
        d["Date"] = date
        data.append(d)
    # Créer le DataFrame
    df = pd.DataFrame(data)
    df.set_index('Ticker', inplace=True)
    return df





In [3]:
def get_data_today():
    rows = get_info(url_price)
    columns = [('Stock Description', 1, 'string'), ('Price J-1', 3, 'numeric'), ('Price', 5, 'numeric')]
    df = get_col_from_rows(rows, columns)
    
    rows2 = get_info(url_capitalisation)
    columns2 = [('Stock Description', 1, 'string'), ("Nombre de titres", 2, "numeric"),
            ("Cours du jour", 3, "numeric"), ("Capitalisation flottante", 4, "numeric"),
            ("Capitalisation globale", 5, "numeric"), ("Capitalisation globale (%)", 6, "numeric")]
    df2 = get_col_from_rows(rows2, columns2)
    
    rows3 = get_info(url_per)
    columns3 = [('Stock Description', 1, 'string'), ("Nombre de titres échangés", 2, "numeric"),
            ("Valeur échangée", 3, "numeric"), ("PER", 4, "numeric"),
            ("valeur globale échangée %", 5, "numeric")]
    df3 = get_col_from_rows(rows3, columns3)
    
    return df, df2, df3

def get_prices_freq(intervalle=15, nb_time=20, end_time=datetime.now().replace(hour=16, minute=30, second=0, microsecond=0)):
    
    if datetime.now() > end_time:
        print(datetime.today().strftime('%Y-%m-%d  :  %I:%M:%Ss PM'), " -  It's past 4:30 PM.")
        return
    nb = 0
    while datetime.now() <= end_time and nb < nb_time :
        df_price, df_cap, df_per = get_data_today()
        d = datetime.today().strftime('%Y-%m-%d-%Ih%Mm%Ss')
        df.to_csv(f'../Stocks_Prices/Continous_Prices/brvm_last_prices_{d}.csv')
        df.to_csv(f'../Stocks_Prices/Continous_PER/brvm_last_per_{d}.csv')
        df.to_csv(f'../Stocks_Prices/Continous_Cap/brvm_last_cap_{d}.csv')

        time.sleep(int(1))  # 15 minutes in seconds
        nb += 1



In [13]:
df_price, df_cap, df_per = get_data_today()
df_price["Ticker"] = list(df_price.index)
df_insertion = df_price[["Date", "Price"]]


In [14]:
df_insertion = df_insertion.reset_index()


In [17]:
insert_dataframe(df_insertion, "stock_prices")

In [15]:
df_insertion.head()


,Ticker,Date,Price
0,ABJC,2025-04-18,1780.0
1,BICC,2025-04-18,15500.0
2,BNBC,2025-04-18,1000.0
3,BOAB,2025-04-18,4690.0
4,BOABF,2025-04-18,3995.0


In [16]:
validate_dataframe(df_insertion, ["Date", "Ticker", "Price"])

✅ DataFrame is valid and ready for insertion.


,Ticker,Date,Price
0,ABJC,2025-04-18,1780.0
1,BICC,2025-04-18,15500.0
26,SCRC,2025-04-18,990.0
27,SDCC,2025-04-18,6250.0
28,SDSC,2025-04-18,1330.0
29,SEMC,2025-04-18,700.0
30,SGBC,2025-04-18,22000.0
31,SHEC,2025-04-18,920.0
32,SIBC,2025-04-18,4795.0
33,SICC,2025-04-18,3550.0


In [19]:
get_prices_from_db("BNBC")

ValueError: 3 columns passed, passed data had 4 columns

In [7]:
url2 = "https://www.investing.com/equities/nei-ceda-ci-historical-data"
r = requests.get(url2, verify=False)
soup = BeautifulSoup(r.content, 'html.parser')
rows = soup.select('table tbody tr')

In [10]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd

# Set up headless Chrome
options = webdriver.ChromeOptions()
options.add_argument("--headless")  # run in background
options.add_argument("--disable-gpu")
driver = webdriver.Chrome(service=Service(), options=options)

url = "https://www.investing.com/equities/nei-ceda-ci-historical-data"
driver.get(url)

# Accept cookies if prompted
try:
    WebDriverWait(driver, 5).until(
        EC.element_to_be_clickable((By.ID, "onetrust-accept-btn-handler"))
    ).click()
except:
    pass

# Wait for the historical data table to load
try:
    table = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "table.genTbl.closedTbl.historicalTbl"))
    )
except:
    print("Table not found.")
    driver.quit()
    exit()

# Extract rows
rows = table.find_elements(By.TAG_NAME, "tr")
data = []
for row in rows[1:]:  # skip header
    cols = row.find_elements(By.TAG_NAME, "td")
    if len(cols) == 7:
        data.append([col.text for col in cols])

driver.quit()

# Build DataFrame
df = pd.DataFrame(data, columns=["Date", "Price", "Open", "High", "Low", "Vol.", "Change %"])
print(df.head())


Table not found.


NameError: name 'table' is not defined

In [ ]:
# === PARAMÈTRES ===
TICKERS = ['ORGT', 'PALC', 'STAC', 'CFAC', 'ABJC', 'SIBC', 'ETIT', 'BOAB', 'UNLC']
URL = 'https://www.brvm.org/fr/cours-actions'
HIST_FILE = 'brvm_prices_history.csv'
WINDOW = 20  # jours pour la moyenne mobile
DATE_TODAY = datetime.today().strftime('%Y-%m-%d')


# === SCRAPER BRVM ===
def get_last_prices():
    r = requests.get(URL)
    soup = BeautifulSoup(r.content, 'html.parser')
    rows = soup.select('table tbody tr')
    data = {}
    for row in rows:
        cols = row.find_all('td')
        if len(cols) < 5:
            continue
        code = cols[0].text.strip()
        if code in TICKERS:
            last_price = float(cols[2].text.strip().replace(',', ''))
            data[code] = last_price
    return data


# === MISE À JOUR HISTORIQUE ===
def update_history(new_data):
    if os.path.exists(HIST_FILE):
        df = pd.read_csv(HIST_FILE, index_col='Date', parse_dates=True)
    else:
        df = pd.DataFrame()
    new_row = pd.DataFrame([new_data], index=[DATE_TODAY])
    df = pd.concat([df, new_row])
    df = df.sort_index()
    df.to_csv(HIST_FILE)
    return df


# === CALCUL Z-SCORE ===
def compute_z_scores(df, window=WINDOW):
    z_scores = {}
    for col in df.columns:
        prices = df[col].dropna()
        if len(prices) < window:
            continue
        ma = prices.rolling(window).mean()
        std = prices.rolling(window).std()
        z = (prices - ma) / std
        latest_z = z.iloc[-1]
        z_scores[col] = latest_z
    return pd.DataFrame.from_dict(z_scores, orient='index', columns=['z_score']).sort_values(by='z_score')


# === MAIN ===
if __name__ == "__main__":
    print("📡 Scraping des prix BRVM...")
    prices_today = get_last_prices()

    print("🧾 Mise à jour de l’historique...")
    df_hist = update_history(prices_today)

    print("📊 Calcul des z-scores...")
    z_df = compute_z_scores(df_hist)

    print("\n📉 Actions sous-évaluées (z-score < -1) :")
    sous_evaluees = z_df[z_df['z_score'] < -1]
    print(sous_evaluees if not sous_evaluees.empty else "Aucune action sous-évaluée aujourd’hui.")


In [ ]:
import pandas as pd
import numpy as np

# === PARAMÈTRES ===
HIST_FILE = 'brvm_prices_history.csv'
MIN_Z = -1
MIN_RETURN_5J = 0.01
MIN_RETURN_10J = 0.01
MAX_PRICE = 10000

# Charger données
df = pd.read_csv(HIST_FILE, index_col='Date', parse_dates=True)
df = df.dropna(axis=1, how='any')  # On garde uniquement les tickers complets

results = []

for ticker in df.columns:
    prices = df[ticker]
    if len(prices) < 20:
        continue

    current_price = prices.iloc[-1]
    mean_20 = prices[-20:].mean()
    std_20 = prices[-20:].std()
    z_score = (current_price - mean_20) / std_20

    return_5j = (prices.iloc[-1] - prices.iloc[-6]) / prices.iloc[-6]
    return_10j = (prices.iloc[-1] - prices.iloc[-11]) / prices.iloc[-11]

    if (
        z_score < MIN_Z and
        return_5j > MIN_RETURN_5J and
        return_10j > MIN_RETURN_10J and
        current_price <= MAX_PRICE
    ):
        results.append({
            'Ticker': ticker,
            'Prix actuel': round(current_price),
            'Z-score': round(z_score, 2),
            'Trend 5j (%)': round(return_5j * 100, 2),
            'Trend 10j (%)': round(return_10j * 100, 2),
            'OK': '✅'
        })

# Résultat
df_filtered = pd.DataFrame(results)
df_filtered = df_filtered.sort_values(by='Z-score')

print("\n🎯 Actions filtrées pour achat :")
print(df_filtered if not df_filtered.empty else "Aucune action ne remplit les critères.")
